# Bayesian Updating with Conjugate Priors (Beta-Binomial)

We model next-day return **direction** as a Bernoulli random variable.

- Observation: $y_t = 1$ if $r_{t+1} > 0$, else $0$
- Likelihood: $y_t \sim \text{Bernoulli}(p)$
- Prior: $p \sim \text{Beta}(\alpha_0, \beta_0)$
- Posterior: $p | y_{1:n} \sim \text{Beta}(\alpha_0 + \sum y_i, \beta_0 + n - \sum y_i)$

The posterior mean $\mathbb{E}[p|\cdot]$ becomes a probability forecast and a trading signal.


# Chapter 10 Bayesian ML (Predictive): Conjugate Priors

These notebooks mirror the *methods* highlighted in
`ml_finance_thoery/machine-learning-for-trading/10_bayesian_machine_learning/README.md`
and apply them to the local `dataset/cleaned/` asset universe to produce
out-of-sample predictions and a backtest using the same **vectorized** engine
used by the `notebooks/ML_Linear_Models_*` notebooks.


In [4]:
from __future__ import annotations

import math
from pathlib import Path
import sys

import numpy as np
import pandas as pd

SEED = 42
rng = np.random.default_rng(SEED)

def find_project_root(start: Path) -> Path:
    p = start.resolve()
    for _ in range(10):
        if (p / 'src').exists() and (p / 'dataset').exists():
            return p
        p = p.parent
    raise RuntimeError(f'Could not find project root from: {start!s}')

PROJECT_ROOT = find_project_root(Path.cwd())
CLEANED_DIR = PROJECT_ROOT / 'dataset' / 'cleaned'

# Ensure `src/` is on sys.path so `backtester` is importable
src_dir = PROJECT_ROOT / 'src'
if str(src_dir) not in sys.path:
    sys.path.append(str(src_dir))


In [5]:
# Load close prices for all assets
from backtester.data import load_cleaned_assets, align_close_prices

files = sorted([p for p in CLEANED_DIR.glob('Asset_*.csv')])
symbols = [p.stem for p in files]
assets_ohlcv = load_cleaned_assets(symbols=symbols, cleaned_dir=str(CLEANED_DIR))
close_prices = align_close_prices(assets_ohlcv).sort_index()

rets = close_prices.pct_change()
y_up_fwd = (rets.shift(-1) > 0).astype(int)

# Time split: align boundaries like ML_linear_models notebooks
TRAIN_YEARS = 7
VAL_MONTHS = 18
TEST_MONTHS = 18

def align_to_trading_date(index: pd.DatetimeIndex, ts: pd.Timestamp) -> pd.Timestamp:
    pos = int(index.searchsorted(ts, side='left'))
    if pos >= len(index):
        return pd.Timestamp(index[-1])
    return pd.Timestamp(index[pos])

idx = pd.DatetimeIndex(close_prices.index).sort_values()
end = pd.Timestamp(idx[-1])
raw_test_start = end - pd.DateOffset(months=TEST_MONTHS)
raw_val_start = raw_test_start - pd.DateOffset(months=VAL_MONTHS)
raw_train_start = raw_val_start - pd.DateOffset(years=TRAIN_YEARS)
test_start = align_to_trading_date(idx, pd.Timestamp(raw_test_start))
val_start = align_to_trading_date(idx, pd.Timestamp(raw_val_start))
train_start = align_to_trading_date(idx, pd.Timestamp(raw_train_start))
print('aligned boundaries:', train_start, val_start, test_start, end)

# Conjugate updating (rolling window to stay adaptive)
ALPHA0 = 1.0
BETA0 = 1.0
LOOKBACK_DAYS = 252

# Posterior mean estimate per day/asset
p_up = pd.DataFrame(index=close_prices.index, columns=close_prices.columns, dtype=float)
for a in close_prices.columns:
    y = y_up_fwd[a].dropna().astype(int)
    # For each date, count successes in the trailing window ending at t-1
    for t in range(len(y.index)):
        dt = y.index[t]
        start = max(0, t - LOOKBACK_DAYS)
        window = y.iloc[start:t]  # exclude current day to avoid leakage
        s = int(window.sum())
        n = int(window.shape[0])
        a_post = ALPHA0 + s
        b_post = BETA0 + (n - s)
        p_up.loc[dt, a] = a_post / (a_post + b_post)

# Signal for backtest: center around 0
signal = (p_up - 0.5).loc[test_start:end]
pred_matrix = signal.dropna(how='all').fillna(0.0)


aligned boundaries: 2016-01-25 00:00:00 2023-01-17 00:00:00 2024-07-16 00:00:00 2026-01-16 00:00:00


## Backtest
We reuse the same *vectorized* backtest engine and portfolio construction
style as the `notebooks/ML_Linear_Models_*` notebooks:

- weekly rebalance
- select top-K assets by prediction (must be positive)
- 1/N equal-weight and MPT variants


In [6]:
from IPython.display import display
from bokeh.io import output_notebook, show

from backtester.data import load_cleaned_assets, align_close_prices
from backtester.engine import BacktestConfig, run_backtest
from backtester.report import compute_backtest_report
from backtester.bokeh_plots import build_interactive_portfolio_layout
from backtester.portfolio import equal_weight, optimize_mpt

output_notebook()

if 'pred_matrix' not in globals():
    raise RuntimeError('Expected `pred_matrix` (index=date, columns=Asset_ID) to exist')

# IMPORTANT: performance metrics should start when the model has signals.
# Keep the original prediction date range before reindexing to market data.
pred_range = pd.DatetimeIndex(pred_matrix.index).sort_values()
if pred_range.empty:
    raise RuntimeError('pred_matrix has empty index')
bt_start = pd.Timestamp(pred_range[0])
bt_end = pd.Timestamp(pred_range[-1])

bt_assets = sorted([str(c) for c in pred_matrix.columns.tolist()])
assets_ohlcv = load_cleaned_assets(symbols=bt_assets, cleaned_dir=str(CLEANED_DIR))
close_prices = align_close_prices(assets_ohlcv)

# Align prediction matrix to available trade dates
pred_matrix = pred_matrix.reindex(close_prices.index)
# Restrict backtest to the prediction window (avoid 2016-start metrics).
close_prices = close_prices.loc[bt_start:bt_end]
pred_matrix = pred_matrix.loc[bt_start:bt_end]
returns_matrix = close_prices.pct_change().fillna(0.0)

market_df = pd.DataFrame({
    'Open': pd.concat([df['Open'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'High': pd.concat([df['High'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Low': pd.concat([df['Low'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Close': pd.concat([df['Close'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Volume': pd.concat([df['Volume'] for df in assets_ohlcv.values()], axis=1).sum(axis=1),
}).sort_index().loc[bt_start:bt_end]

REBALANCE_FREQ = 'W'
TOP_K = min(20, len(bt_assets))
LOOKBACK_DAYS = 126

def build_weights_from_predictions(pred_matrix: pd.DataFrame, *, pm_style: str) -> pd.DataFrame:
    rebal_dates = set(pd.Series(pred_matrix.index, index=pred_matrix.index).resample(REBALANCE_FREQ).last().dropna().tolist())
    w_last = pd.Series(0.0, index=bt_assets)
    rows = []
    for dt in pred_matrix.index:
        if dt in rebal_dates:
            row = pred_matrix.loc[dt].dropna().sort_values(ascending=False)
            top = row.head(TOP_K)
            candidates = [a for a, v in top.items() if np.isfinite(v) and float(v) > 0.0]
            if not candidates:
                w_last = pd.Series(0.0, index=bt_assets)
            else:
                if pm_style == '1N':
                    w_dict = equal_weight(candidates)
                elif pm_style == 'MPT':
                    w_dict = optimize_mpt(returns_matrix, candidates, dt, lookback_days=LOOKBACK_DAYS)
                else:
                    raise ValueError(f'Unknown pm_style: {pm_style!r}')
                w_last = pd.Series(0.0, index=bt_assets)
                for a, w in w_dict.items():
                    w_last[str(a)] = float(w)
        rows.append(w_last)
    return pd.DataFrame(rows, index=pred_matrix.index, columns=bt_assets).fillna(0.0)

cfg = BacktestConfig(initial_equity=1_000_000.0, transaction_cost_bps=5.0, mode='vectorized')

compare_rows = []
results = {}
for pm_style in ['1N', 'MPT']:
    w = build_weights_from_predictions(pred_matrix, pm_style=pm_style)
    res = run_backtest(close_prices, w, config=cfg)
    rpt = compute_backtest_report(result=res, close_prices=close_prices)
    results[pm_style] = (w, res, rpt)
    compare_rows.append({
        'style': pm_style,
        'Total Return [%]': float(rpt['Total Return [%]']),
        'CAGR [%]': float(rpt['CAGR [%]']),
        'Sharpe': float(rpt['Sharpe']),
        'Max Drawdown [%]': float(rpt['Max Drawdown [%]']),
    })
compare = pd.DataFrame(compare_rows).sort_values('Total Return [%]', ascending=False).reset_index(drop=True)
display(compare)

BASE_TITLE = 'Bayes Conjugate Priors (Beta-Binomial)'
for pm_style in ['1N', 'MPT']:
    w, res, rpt = results[pm_style]
    title = BASE_TITLE + ' - ' + pm_style
    display(rpt.to_frame(title))
    layout = build_interactive_portfolio_layout(
        market_ohlcv=market_df,
        equity=res.equity,
        returns=res.returns,
        weights=res.weights,
        turnover=res.turnover,
        costs=res.costs,
        close_prices=close_prices,
        title=title,
    )
    show(layout)


Loading BokehJS ...

,style,Total Return [%],CAGR [%],Sharpe,Max Drawdown [%]
0,1N,34.842081,22.053517,1.483627,-12.921751
1,MPT,16.694765,10.841123,0.750565,-13.418566


,Bayes Conjugate Priors (Beta-Binomial) - 1N
Start,2024-07-16 00:00:00
End,2026-01-16 00:00:00
Duration,549 days 00:00:00
Initial Equity,1000000.0
Final Equity,1348420.807267
Equity Peak,1348420.807267
Total Return [%],34.842081
CAGR [%],22.053517
Volatility (ann) [%],14.069031
Sharpe,1.483627


,Bayes Conjugate Priors (Beta-Binomial) - MPT
Start,2024-07-16 00:00:00
End,2026-01-16 00:00:00
Duration,549 days 00:00:00
Initial Equity,1000000.0
Final Equity,1166947.652733
Equity Peak,1166947.652733
Total Return [%],16.694765
CAGR [%],10.841123
Volatility (ann) [%],15.231201
Sharpe,0.750565
